In [254]:
import numpy as np
import pandas as pd

from statsmodels.sandbox.regression.gmm import GMM
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, recall_score, precision_score, f1_score

**PART 1 (20%)**

In [255]:
input_table = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Predictive Modelling | MBAN 5110 U /Midterm/midterm_partone.csv')

In [256]:
input_table.head()

,Constant,Stock Change,Inventory Turnover,Operating Profit,Interaction Effect,Current Ratio,Quick Ratio,Debt Asset Ratio
0,1,0.870332,1.795946,0.115846,0.208053,1.672527,0.255171,0.473317
1,1,-0.047347,1.395501,0.436967,0.609788,1.637261,0.221763,0.489967
2,1,0.001176,1.664563,0.541016,0.900555,1.640619,0.189141,0.374269
3,1,-0.901200,1.605738,0.539399,0.866133,1.436221,0.131944,0.224399
4,1,-0.176353,1.591451,0.539938,0.859285,1.433140,0.183095,0.213446


In [257]:
input_table.describe()

,Constant,Stock Change,Inventory Turnover,Operating Profit,Interaction Effect,Current Ratio,Quick Ratio,Debt Asset Ratio
count,1696.0,1696.000000,1696.000000,1696.000000,1696.000000,1696.000000,1696.000000,1696.000000
mean,1.0,-0.012977,24.633183,0.255719,6.584336,1.972308,1.069610,0.150254
std,0.0,0.490888,37.044608,0.590330,42.544580,1.509598,1.161417,0.275665
min,1.0,-2.366261,0.996475,-5.989709,-411.751139,0.202040,0.035457,0.000000
25%,1.0,-0.258291,3.733259,0.151177,0.646555,1.079045,0.366561,0.000000
50%,1.0,-0.005839,6.372805,0.244090,1.596459,1.585408,0.694119,0.037937
75%,1.0,0.234015,29.433657,0.338010,5.298054,2.364317,1.258383,0.236709
max,1.0,4.138063,240.046041,7.450718,884.467955,13.026268,10.501619,3.675735


## Original GMM model from class

In [258]:
y_vals  = np.array(input_table["Stock Change"])
x_vals  = np.array(input_table[["Inventory Turnover","Operating Profit","Interaction Effect"]])
iv_vals = np.array(input_table[["Current Ratio","Quick Ratio","Debt Asset Ratio"]])

class gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3 = params
        endog = self.endog
        exog = self.exog
        inst = self.instrument   

        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,0] 
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,1] 
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2]) * inst[:,2] 

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g


beta0 = np.array([0.1, 0.1, 0.1, 0.1])
res = gmm(endog = y_vals, exog = x_vals, instrument = iv_vals, k_moms=6, k_params=4).fit(beta0)

res.summary()

Optimization terminated successfully.
         Current function value: 0.000046
         Iterations: 8
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.000373
         Iterations: 7
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 9
         Gradient evaluations: 9
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 5
         Function evaluations: 11
         Gradient evaluations: 11
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 0
         Function evaluations: 1
         Gradient evaluations: 1


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.6317
Model:                            gmm   Prob (Hansen J):                 0.729
Method:                           GMM                                         
Date:                Fri, 10 Nov 2023                                         
Time:                        14:07:46                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0200      0.021     -0.964      0.335      -0.061       0.021
p 1            0.0011      0.001      1.843      0.065   -6.89e-05       0.002
p 2           -0.1071      0.032     -3.370      0.001      -0.169      -0.045
p 3            0.0011      0.000      2.760      0.006       0.000       0.002
==============================================================================
"""

## 1. Update the GMM model that we discussed in class by incorporating the 𝛿 term to the instrumental-variable moment expressions.

In [259]:
y_vals  = np.array(input_table["Stock Change"])
x_vals  = np.array(input_table[["Inventory Turnover","Operating Profit","Interaction Effect"]])
iv_vals = np.array(input_table[["Current Ratio","Quick Ratio","Debt Asset Ratio"]])

class gmm(GMM):
    def momcond(self, params):
        p0, p1, p2, p3, delta = params #Added in the delta term in the params
        endog = self.endog
        exog = self.exog
        inst = self.instrument   
#Added the delta term to the error calculations.
        error0 = endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2] - delta * 1
        error1 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2] - delta * 1) * exog[:,1]
        error2 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2] - delta * 1) * exog[:,2]
        error3 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2] - delta * 1) * inst[:,0]
        error4 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2] - delta * 1) * inst[:,1]
        error5 = (endog - p0 - p1 * exog[:,0] - p2 * exog[:,1] - p3 * exog[:,2] - delta * 1) * inst[:,2]

        g = np.column_stack((error0, error1, error2, error3, error4, error5))
        return g

beta0 = np.array([0.1, 0.1, 0.1, 0.1, 0.1])
res = gmm(endog=y_vals, exog=x_vals, instrument=iv_vals, k_moms=6, k_params=5).fit(beta0) #Updated the k_params to include delta

res.summary()


Optimization terminated successfully.
         Current function value: 0.000046
         Iterations: 10
         Function evaluations: 13
         Gradient evaluations: 13
Optimization terminated successfully.
         Current function value: 0.000373
         Iterations: 7
         Function evaluations: 12
         Gradient evaluations: 12
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 6
         Function evaluations: 10
         Gradient evaluations: 10
Optimization terminated successfully.
         Current function value: 0.000372
         Iterations: 1
         Function evaluations: 3
         Gradient evaluations: 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                                 gmm Results                                  
==============================================================================
Dep. Variable:                      y   Hansen J:                       0.6317
Model:                            gmm   Prob (Hansen J):                 0.427
Method:                           GMM                                         
Date:                Fri, 10 Nov 2023                                         
Time:                        14:07:46                                         
No. Observations:                1696                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
p 0           -0.0100    1.5e+06  -6.68e-09      1.000   -2.94e+06    2.94e+06
p 1            0.0011      0.001      1.843      0.065    -6.9e-05       0.002
p 2           -0.1071      0.032     -3.370      0.001      -0.169      -0.045
p 3            0.0011      0.000      2.760      0.006       0.000       0.002
p 4           -0.0100    1.5e+06  -6.68e-09      1.000   -2.94e+06    2.94e+06
==============================================================================
"""

**Standard Error**

In the initial model, the standard errors are all quite small. With the inclusion of the delta variable, the standard error of p0 becomes very large, as well as the standard error of p4. This suggests that estimating the value of this coeffecients becomes very unstable/inconsistent. 

**Statistical significance (Z & P Scores)**

Although p1,p2 & p3 values did not change, p0 went from having some significance (p score of 0.335, z score -0.964), to having no significance (z-score of -6.68e-09, p-score of 1.000). The delta value p4 has the exact same z and p score as p0, suggesting no statistical significance on the dependent variable.


## PART 2 (80%) ##

In [260]:
df = pd.read_csv('/Users/mikeredshaw/Documents/Schulich MBAN/Predictive Modelling | MBAN 5110 U /Midterm/midterm_parttwo.csv')

In [261]:
df.head()

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
0,1,Low,No dependent,Very low,Very low,Married,Positive
1,2,Low,No dependent,Very low,Very low,Single,Positive
2,1,Low,No dependent,Very low,Very low,Single,Positive
3,3,Low,No dependent,Very low,Very low,Married,Positive
4,3,Low,No dependent,Very low,Very low,Single,Negative


In [262]:
df.describe()

,Years of Education after High School
count,8081.000000
mean,2.608588
std,1.571835
min,0.000000
25%,1.000000
50%,3.000000
75%,3.000000
max,7.000000


In [263]:
df_unchanged = df.copy()

I want to convert all the categorical variables into numerical to be used in the regression model. I'll do this quickly by hand for easy view of the meaning of the new labels.

In [264]:
print(df['Requested Credit Amount'].unique())
print(df['Number of Dependents'].unique())
print(df['Monthly Income'].unique())
print(df['Monthly Expense'].unique())
print(df['Marital Status'].unique())
print(df['Credit Rating'].unique())

['Low' 'Medium' 'High']
['No dependent' 'Less than 2' 'More than 2']
['Very low' 'Low' 'Moderate' 'High' 'Very High']
['Very low' 'Low' 'Moderate' 'High' 'Very high']
['Married' 'Single' 'Not specified']
['Positive' 'Negative']


In [265]:
df['Requested Credit Amount'] = df['Requested Credit Amount'].map({'Low': 1, 'Medium': 2, 'High': 3})
df['Number of Dependents'] = df['Number of Dependents'].map({'No dependent': 1, 'Less than 2': 2, 'More than 2': 3})
df['Monthly Income'] = df['Monthly Income'].map({'Very low':1, 'Low':2, 'Moderate': 3, 'High': 4, 'Very High': 5})
df['Monthly Expense'] = df['Monthly Expense'].map({'Very low':1, 'Low':2, 'Moderate': 3, 'High': 4, 'Very high': 5})
df['Marital Status'] = df['Marital Status'].map({'Married':1, 'Single':2, 'Not specified': 3})
df['Credit Rating'] = df['Credit Rating'].map({'Positive':1, 'Negative':0})


In [266]:
print(df['Requested Credit Amount'].unique())
print(df['Number of Dependents'].unique())
print(df['Monthly Income'].unique())
print(df['Monthly Expense'].unique())
print(df['Marital Status'].unique())
print(df['Credit Rating'].unique())

[1 2 3]
[1 2 3]
[1 2 3 4 5]
[1 2 3 4 5]
[1 2 3]
[1 0]


In [267]:
df.head()

,Years of Education after High School,Requested Credit Amount,Number of Dependents,Monthly Income,Monthly Expense,Marital Status,Credit Rating
0,1,1,1,1,1,1,1
1,2,1,1,1,1,2,1
2,1,1,1,1,1,2,1
3,3,1,1,1,1,1,1
4,3,1,1,1,1,2,0


**Divide the dataset equally into two as training (50%) and test (50%) sets. Use the training set to fit a logistic regression model, where the credit rating is the dependent variable. Apply the model to the test set, and report the confusion matrix, recall, precision, and F1 score values.**

In [268]:
train, test = train_test_split(df, test_size=0.5, random_state=42)

X_train = train.drop('Credit Rating', axis=1)
y_train = train['Credit Rating']
X_test = test.drop('Credit Rating', axis=1)
y_test = test['Credit Rating']

In [269]:
print(y_test.value_counts())
print(y_train.value_counts())

1    3464
0     577
Name: Credit Rating, dtype: int64
1    3471
0     569
Name: Credit Rating, dtype: int64


In [270]:
model = LogisticRegression(random_state=0, max_iter = 1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

In [271]:
conf_matrix = confusion_matrix(y_test, y_pred)
recall = recall_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print("Confusion Matrix:\n", conf_matrix)
print("\nRecall:", recall)
print("Precision:", precision)
print("F1 Score:", f1)


Confusion Matrix:
 [[   0  577]
 [   0 3464]]

Recall: 1.0
Precision: 0.8572135609997525
F1 Score: 0.9231179213857428


Here, we can see that the model is predicting positive for credit score every single time. Recall is a perfect 1 as it's obviously not missing any predictions on true positives. Precision is also still quite high since 85% of the samples are positives, meaning 85% of the time the model predicts positive, it's correct, and 15% of the time it predicts positive, it's a false positive. This model isn't very good, as it would label every single customer as having a positive credit score.

Lets now set the threshold of predicting positive to 15%. I'll do this first by using the logistic regression model to predict the probability of a sample being a positive credit score, and then use the top 15% of probability samples to predict as approve, and the rest as deny.

In [272]:
y_scores = model.predict_proba(X_test)[:, 1]  #Since I want the probabilities of a person being labeled as positive credit score, I will run the positive column through the predict proba function.
threshold_15 = np.percentile(y_scores, 85)  # I want a 15% threshold, so to set this, I'll use numpy percentile function, setting the cut off at the 85th percentile of the probability.
print(threshold_15)

0.8789607304891762


The threshold value would be 0.8789607304891762 predicted probability of being categorized as having positive credit score. This means that we can set the required probability to be above this in order to ensure only 15% of applicants get approved.

In [273]:
y_pred_15 = (y_scores >= threshold_15).astype(int) # Setting y pred to be positive if the predicted probabilities from the regression model is above our threshold.

conf_matrix_15 = confusion_matrix(y_test, y_pred_15)
recall_15 = recall_score(y_test, y_pred_15)
precision_15 = precision_score(y_test, y_pred_15)
f1_15 = f1_score(y_test, y_pred_15)

print("Confusion Matrix:\n", conf_matrix_15)
print("\nRecall:", recall_15)
print("Precision:", precision_15)
print("F1 Score:", f1_15)


Confusion Matrix:
 [[ 499   78]
 [2922  542]]

Recall: 0.1564665127020785
Precision: 0.8741935483870967
F1 Score: 0.2654260528893242


We can see that making the model more conservative, restricting it to only 15% approval has changed the performance significantly. It increased the true negatives the model predicted from 0 to 499, and decreased the false positives from 577 to 78, showcasing that the model has become much more conservative with predicting positive. 

We also see that false negatives has increased significantly with a sharp decrease in true positive predictions. This coincides with the model being limited to only predicting positives on 15% of the samples. 

Since roughly 85% of applications actually have positive credit scores, forcing the model to only predict 15% as positive would significantly impact these results.